<a href="https://colab.research.google.com/github/AntonBespalov/FastaiModelToCpp/blob/master/TorchScriptOfFastAIModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
!curl -s https://course.fast.ai/setup/colab | bash

Updating fastai...
Done.


In [0]:
import PIL
from PIL import Image

import cv2

import fastai
from fastai.vision import *

import torch
import torchvision
from torchvision import transforms

In [4]:
print(f'opencv version: {cv2.__version__}')
print(f'fastai version: {__version__}')


opencv version: 3.4.3
fastai version: 1.0.59


In [5]:
defaults.device = torch.device('cpu'); defaults.device

device(type='cpu')

In [0]:
size = 28
image_path = 'test_img.png'

Create a random image with OpenCV.

In [7]:
image_array = np.random.randn(28, 28, 3).astype(np.uint8)
cv2.imwrite(image_path, image_array)

True

In [0]:
path = untar_data(URLs.MNIST_TINY)
data = ImageDataBunch.from_folder(path, size=size)

In [0]:
learn = cnn_learner(data, models.resnet18)

The model is supposed to be trained here. I'm skipping this step, because it doesn't matter to our task.

In [11]:
pred = learn.predict(open_image(image_path)); pred[2]

tensor([0.7567, 0.2433])

In [0]:
traced_script_module = torch.jit.trace(learn.model, torch.rand(1, 3, size, size))

In [0]:
traced_script_module.save("traced_model.pt")

Let's make sure on the Python side, that we have the same predictions from predict() method we called for our Learner and from forward() method we called for TorchScript representation.

In [0]:
opencvImage = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
opencvImage = cv2.cvtColor(np.array(opencvImage), cv2.COLOR_RGB2BGR)

imageTensor = torch.from_numpy(opencvImage)
imageTensor = imageTensor.float().div_(255)
imageTensor = imageTensor.permute(2, 0, 1)

In [15]:
out = traced_script_module.forward(imageTensor[None]);
out_softmax = out.softmax(-1); out_softmax

tensor([[0.7567, 0.2433]], grad_fn=<SoftmaxBackward>)

The predictions are equal.